# Writing Containers to a tabular format

The `TableWriter`/`TableReader` sub-classes allow you to write a `ctapipe.core.Container` class and its meta-data to an output table. They treat the `Field`s in the `Container` as columns in the output, and automatically generate a schema.  Here we will go through an example of writing out data and reading it back with *Pandas*, *PyTables*, and a `ctapipe.io.TableReader`:

In this example, we will use the `HDF5TableWriter`, which writes to HDF5 datasets using *PyTables*. Currently this is the only implemented TableWriter.

Caveats to think about:
* vector columns in Containers *can* be written, but some lilbraries like Pandas can not read those (so you must use pytables or astropy to read outputs that have vector columns)
* units are stored in the table metadata, but some libraries like Pandas ignore them and all other metadata

## Create some example Containers

In [ ]:
from ctapipe.io import HDF5TableWriter
from ctapipe.core import Container, Field
from astropy import units as u
import numpy as np

In [ ]:
class VariousTypesContainer(Container):
    
    a_int = Field(int, 'some int value')
    a_float = Field(float, 'some float value with a unit', unit=u.m)
    a_bool = Field(bool, 'some bool value')
    a_np_int = Field(np.int, 'a numpy int')
    a_np_float = Field(np.float, 'a numpy float')
    a_np_bool = Field(np.bool, 'np.bool')

let's also make a dummy stream (generator) that will create a series of these containers

In [ ]:
def create_stream(n_event):
    
    data = VariousTypesContainer()
    for i in range(n_event):
        
        data.a_int = int(i)
        data.a_float = float(i) * u.cm # note unit conversion will happen
        data.a_bool = (i % 2) == 0
        data.a_np_int = np.int(i)
        data.a_np_float = np.float(i)
        data.a_np_bool = np.bool((i % 2) == 0)
        
        yield data

In [ ]:
for data in create_stream(2):
        
    for key, val in data.items():
        
        print('{}: {}, type : {}'.format(key, val, type(val)))

## Writing the Data (and good practices)

### How not to do it:

In [ ]:
h5_table = HDF5TableWriter('container.h5', group_name='data')

for data in create_stream(10):
    
    h5_table.write('table', data)

h5_table.close()

In that case the file is not garenteed to close properly for instance if one does a mistake in the for loop. Let's just add a stupid mistake and see what happens.

In [ ]:
try:
    h5_table = HDF5TableWriter('container.h5', group_name='data')

    for data in create_stream(10):

        h5_table.write('table', data)
        0/0  # cause an error
        
    h5_table.close()
except Exception as err:
    print("FAILED!", err)

Now the file did not close properly. So let's try to correct the mistake and execute the code again.

In [ ]:
try:
    h5_table = HDF5TableWriter('container.h5', group_name='data')

    for data in create_stream(10):

        h5_table.write('table', data)
        0/0  # cause an error
    h5_table.close()
except Exception as err:
    print("FAILED!", err)

Ah it seems that the file did not close! Now I am stuck. Maybe I should restart the kernel? ahh no I don't want to loose everything. Can I just close it ?

In [ ]:
h5_table.close()

It worked!

### Better to use context management!

In [ ]:
try:
    with HDF5TableWriter('container.h5', group_name='data') as h5_table:

        for data in create_stream(10):

            h5_table.write('table', data)
            0/0
except Exception as err:
    print("FAILED:", err)
print('Done')

In [ ]:
!ls container.h5

## Appending new Containers

To append some new containers we need to set the writing in append mode by using: 'mode=a'. But let's now first look at what happens if we don't.

In [ ]:
for i in range(2):

    with HDF5TableWriter('container.h5', mode='w', group_name='data_{}'.format(i)) as h5_table:
    
        for data in create_stream(10):
    
            h5_table.write('table', data)

        print(h5_table._h5file)

In [ ]:
!rm container.h5

Ok so the writer destroyed the content of the file each time it opens the file. Now let's try to append some data group to it! (using mode='a')

In [ ]:
for i in range(2):

    with HDF5TableWriter('container.h5', mode='a', group_name='data_{}'.format(i)) as h5_table:
    
        for data in create_stream(10):
    
            h5_table.write('table', data)

        print(h5_table._h5file)

So we can append some data groups. As long as the data group_name does not already exists. Let's try to overwrite the data group : data_1

In [ ]:
try:
    with HDF5TableWriter('container.h5', mode='a', group_name='data_1') as h5_table:
        for data in create_stream(10):
            h5_table.write('table', data)
except Exception as err:
    print("Failed as expected:", err)

Good ! I cannot overwrite my data.

In [ ]:
print(bool(h5_table._h5file.isopen))

## Reading the Data

### Reading the whole table at once:

For this, you have several choices.  Since we used the HDF5TableWriter in this example, we have at least these options avilable:

* Pandas
* PyTables
* Astropy Table

For other TableWriter implementations, others may be possible (depending on format)


#### Reading with Pandas:

Pandas is a convenient way to read the output.  **HOWEVER BE WARNED** that so far Pandas does not support reading the table *meta-data* or *units* for colums, so that information is lost! 

In [ ]:
import pandas as pd

data = pd.read_hdf('container.h5', key='/data_0/table')
data.head()

#### Reading with PyTables

In [ ]:
import tables
h5  = tables.open_file('container.h5')
table = h5.root['data_0']['table']
table

note that here we can still access the metadata

In [ ]:
table.attrs

### Reading one-row-at-a-time:

Rather than using the full-table methods, if you want to read it row-by-row (e.g. to maintain compatibility with an existing event loop), you can use a `TableReader` instance.

The advantage here is that units and other metadata are retained and re-applied

In [ ]:
from ctapipe.io import HDF5TableReader

def read(mode):
    
    print('reading mode {}'.format(mode))

    with HDF5TableReader('container.h5', mode=mode) as h5_table:

        for group_name in ['data_0/', 'data_1/']:

            group_name = '/{}table'.format(group_name)
            print(group_name)

            for data in h5_table.read(group_name, VariousTypesContainer()):

                print(data.as_dict())

In [ ]:
read('r')

In [ ]:
read('r+')

In [ ]:
read('a')

In [ ]:
read('w')